In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal
from typing import List
import ipywidgets as widgets
from IPython.display import display, HTML

def gaussian(x: np.ndarray, x0: float, sigma: float) -> np.ndarray:
    """ Return the normalized Gaussian with standard deviation sigma. """
    c = np.sqrt(2 * np.pi)
    return np.exp(-0.5 * ((x - x0) / sigma) ** 2) / sigma / c

# Dimensions of the lidar image
rows = 80
columns = 1200
zero_array = np.zeros((rows, columns))# Create the array of zeroes that matches the dimensions of the lidar image (columns will indicate time range)

# Range resolution and time resolution given by lidar
range_res = 0.05      # meters
time_res = 0.001      # seconds

# Range array[m]
x = np.linspace(0, rows * range_res, rows)

# Time array[s]
time_array = np.zeros(columns)
for index in range(0, columns, 1):
    time_array[index] = 0.0 + time_res * index

def plot_lidar_image(velocity, slice_index):
    # Insert the Gaussian into the zeros array and visualize
    start = 0           # index, unitless
    end = 1200          # index, unitless
    range_offset = 2    # meters

    # Reset zero_array
    zero_array[:] = 0

    # Generate the Gaussian plug and insert it into the zeroes matrix
    for col_idx in range(start, end):
        zero_array[:, col_idx] = gaussian(x, range_offset + time_array[col_idx] * velocity, 0.05)

    # Setting up the meshgrid for plotting
    a = np.linspace(0, time_res * columns, zero_array.shape[1])
    b = np.linspace(0, range_res * rows, zero_array.shape[0])
    A, B = np.meshgrid(a, b)

    # Plotting
    plt.figure(figsize=(8, 6))
    c = plt.pcolormesh(A, B, zero_array, cmap='jet', shading='auto')  # Using 'jet' colormap
    plt.colorbar(c, label='Backscatter intensity')  # Customize colorbar label
    plt.title('Lidar Image of Simulated Gaussian Plume')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Range (meters)')
    plt.show()

    # Pull sequences from the array to perform the cross-correlation
    sequence1 = zero_array[:, 0]
    sequence2 = zero_array[:, slice_index]

    def compute_correlations(seq1, seq2):
        len_seq1 = len(seq1)                        # Length of the array. Both are of the same length
        len_seq2 = len(seq2)
        cross_corr_result = []                      # Initialize empty list to store result of the cross correlation computation
        norm_correlation = []                       # Initialize empty list to store normalized cross correlation
        for lag in range(-len_seq2 + 1, len_seq1):  # Iterate over all possible lags
            if lag < 0:                             # If the lag is negative
                part_seq1 = seq1[:len_seq1 + lag]   # Extract the beginning portion of seq1
                part_seq2 = seq2[-lag:]             # Extract the ending portion of seq2
            else:                                   # If the lag is zero or positive
                part_seq1 = seq1[lag:]              # Extract the ending portion of seq1
                part_seq2 = seq2[:len(part_seq1)]   # Extract the beginning portion of seq2
            cross_corr_value = np.sum(part_seq1 * part_seq2) # Compute the cross-correlation value
            cross_corr_result.append(cross_corr_value)       # Append the result to the list
            norm_corr = np.sqrt(np.sum(part_seq1**2) * np.sum(part_seq2**2)) # Normalize sequences for NCC computation
            norm_correlation.append(norm_corr)       # Append the result to the list
        cross_corr_result = np.array(cross_corr_result) # Convert to arrays
        norm_correlation = np.array(norm_correlation)   # Convert to arrays
        norm_corr_result = (cross_corr_result / norm_correlation) * 100 ## NCC computation
        return cross_corr_result, norm_corr_result

    def find_max_cross_corr_shift(seq1, seq2):
        cross_corr_result, norm_corr_result = compute_correlations(seq1, seq2)

        # Find the shift with the maximum cross-correlation
        max_index = np.argmax(cross_corr_result)

        # Calculate the shift
        shift = max_index - (len(seq1) - 1)

        return shift, norm_corr_result[max_index], cross_corr_result, norm_corr_result

    # Compute cross-correlation and find the optimal shift
    shift, max_percent_cross_corr, cross_corr_result, norm_corr_result = find_max_cross_corr_shift(sequence1, sequence2)

    # Define the lag array
    lags = np.arange(-len(sequence2) + 1, len(sequence1))
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 12))

    ax1.plot(x, sequence1, label='t', color='b', linewidth=2)
    ax1.set_title('Lidar Pulse at t=t', fontsize=14)
    ax1.set_xlabel('Range (m)', fontsize=12)
    ax1.set_ylabel('Backscatter Intensity', fontsize=12)
    ax1.grid(True)
    ax1.legend()

    ax2.plot(x, sequence2, label=f'(δt: {slice_index})', color='g', linewidth=2)
    ax2.set_title('Lidar Pulse at t=t+δt', fontsize=14)
    ax2.set_xlabel('Range (m)', fontsize=12)
    ax2.set_ylabel('Backscatter Intensity', fontsize=12)
    ax2.grid(True)
    ax2.legend()

    ax3.plot(lags, cross_corr_result, label='Cross Correlation', color='r', linewidth=2)
    ax3.set_title('Cross Correlation Vector of the t and t+δt Signals', fontsize=14)
    ax3.set_xlabel('Lag', fontsize=12)
    ax3.set_ylabel('Cross Correlation Value', fontsize=12)
    ax3.axvline(x=shift, color='k', linestyle='--', label=f'Max correlation at lag: {shift}')
    ax3.grid(True)
    ax3.legend()

    ax4.plot(lags, norm_corr_result, label='Percent Correlation', color='m', linewidth=2)
    ax4.set_title('Percent Correlation', fontsize=14)
    ax4.set_xlabel('Lag', fontsize=12)
    ax4.set_ylabel('Percent Correlation (%)', fontsize=12)
    ax4.axvline(x=shift, color='k', linestyle='--', label=f'Max correlation: {max_percent_cross_corr:.2f}%')
    ax4.grid(True)
    ax4.legend()

    plt.tight_layout()
    plt.show()

    p_shift = shift
    distance = p_shift * range_res
    velocity_calculated = ((-1) * distance) / (slice_index * time_res)
    print(f"The maximum correlation occurs at a lag of {p_shift}")
    print(f"The distance traveled is {distance} meters")
    print(f"The velocity of the gaussian plug is {velocity_calculated:.2f} meters per second")

# Create interactive widgets for velocity and slice index input
velocity_slider = widgets.FloatSlider(min=-5, max=5, step=0.1, value=5.0, description='VELOCITY (m/s)')
slice_index_slider = widgets.IntSlider(min=1, max=1199, step=1, value=10, description='δt (ms)')

# Customize the labels to be capitalized

velocity_slider.style.handle_color = 'lightblue'
slice_index_slider.style.handle_color = 'lightblue'

# Display the widgets
interactive_plot = interactive(plot_lidar_image, velocity=velocity_slider, slice_index=slice_index_slider)
display(interactive_plot)


interactive(children=(FloatSlider(value=5.0, description='VELOCITY (m/s)', max=5.0, min=-5.0, style=SliderStyl…